In [1]:
import numpy as np
import time
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import UpSampling1D
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
import pandas as pd

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)


In [ ]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
print('X:', X)
print('y:', y)

## Parameters

In [2]:
ts_dimension = 1,
dilations = (1, 2, 4, 8, 16),
nb_filters = 20,
kernel_size = 20,
nb_stacks = 1,
padding = 'causal',
dropout_rate = 0.00,
filters_conv1d = 8,
activation_conv1d = 'linear',
latent_sample_rate = 42,
pooler = AveragePooling1D,
lr = 0.001,
conv_kernel_init = 'glorot_normal',
loss = 'logcosh',
use_early_stopping = False,
error_window_length = 128,
verbose = 1

## TCN Model

In [3]:
from tcn import TCN

In [ ]:
i = Input(shape=(1050, 1))

# Put signal through TCN. Output-shape: (batch,sequence length, nb_filters)
tcn_enc = TCN(nb_filters=nb_filters, kernel_size=kernel_size, nb_stacks=1, dilations=[1, 2, 4, 8, 16], 
                padding='causal', use_skip_connections=True, dropout_rate=dropout_rate, return_sequences=True,
                kernel_initializer='glorot_normal', name='tcn-enc')(i)

# Now, adjust the number of channels...
enc_flat = Conv1D(filters=filters_conv1d, kernel_size=1, activation=activation_conv1d, padding='causal')(tcn_enc)

## Do some average (max) pooling to get a compressed representation of the time series (e.g. a sequence of length 8)
enc_pooled = pooler(pool_size=latent_sample_rate, strides=None, padding='causal', data_format='channels_last')(enc_flat)

# If you want, maybe put the pooled values through a non-linear Activation
enc_out = Activation("linear")(enc_pooled)
model_tcn = Model(inputs=[i], outputs=[enc_out])

## Training a neural network to invert feature representations

We should have a short sequence, which we will upsample again and then try to reconstruct the original series

In [ ]:
i = Input(batch_shape=enc_out.shape)
# Now we should have a short sequence, which we will upsample again and then try to reconstruct the original series
dec_upsample = UpSampling1D(size=latent_sample_rate)(enc_out)

dec_reconstructed = TCN(nb_filters=nb_filters, kernel_size=kernel_size, nb_stacks=nb_stacks, dilations=dilations, 
                        padding=padding, use_skip_connections=True, dropout_rate=dropout_rate, return_sequences=True,
                        kernel_initializer=conv_kernel_init, name='tcn-dec')(dec_upsample)

# Put the filter-outputs through a dense layer finally, to get the reconstructed signal
o = Dense(ts_dimension, activation='linear')(dec_reconstructed)

model = Model(inputs=[i], outputs=[o])